In [1]:
from subprocess import Popen, PIPE, CalledProcessError
import re
import platform
import shutil
import os.path
import fileinput

options = {'stdout': PIPE, 'stderr': PIPE, 'bufsize' : 1, 'universal_newlines' : True, 'shell' : False}
if (platform.system() == 'Windows'):
    options['shell'] = True
    #Packages required to generate ssh keys in windows
    from cryptography.hazmat.primitives import serialization as crypto_serialization
    from cryptography.hazmat.primitives.asymmetric import rsa
    from cryptography.hazmat.backends import default_backend as crypto_default_backend

def callPopen(cmd):
    print(cmd)
    with Popen(cmd.split(),**options) as p:
        for line in p.stdout:
            print(line, end='')
        for line in p.stderr:
            print(line, end='')
        if p.returncode != (0 or None):
            raise CalledProcessError(p.returncode, p.args)

def isInstance(name):
    instanceExists=False
    ip=''
    with Popen('gcloud compute instances list'.split(),**options) as p:
        for line in p.stdout:
            if re.match('^{}'.format(name), line):
                instanceExists=True
                ip = line.strip().split()
                ip = ip[4]
        for line in p.stderr:
            print(line, end='')
        if p.returncode != (0 or None):
            raise CalledProcessError(p.returncode, p.args)
        return(instanceExists,ip)
    
            
def text_prepender(filename, text):
    with open(filename, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(text.rstrip('\r\n') + content)

def replace(file,pattern,replace):
    myfile = fileinput.FileInput(file, inplace=True)
    for line in myfile:
        line = re.sub(pattern,
                  replace,
                  line.rstrip())
        
if (platform.system() == 'Windows'):
    def generateSSHKey(username,savePath):
        key = rsa.generate_private_key(
            backend=crypto_default_backend(),
            public_exponent=65537,
            key_size=2048
            )
        private_key = key.private_bytes(
            crypto_serialization.Encoding.PEM,
            crypto_serialization.PrivateFormat.TraditionalOpenSSL,
            crypto_serialization.NoEncryption()
            )
        public_key = key.public_key().public_bytes(
            crypto_serialization.Encoding.OpenSSH,
            crypto_serialization.PublicFormat.OpenSSH
            )
        public_file = os.path.join(savePath,username + '.pub')
        private_file = os.path.join(savePath,username)
        text_file = open(public_file, "w")
        text_file.write(public_key.decode('utf-8') + ' ' + username)
        text_file.close()
        text_file = open(private_file, "w")
        text_file.write(private_key.decode('utf-8'))
        text_file.close()
        print('Successfully created key pair')
            
if (platform.system() == 'Linux'):
    def generateSSHKey(username,savePath):
        p = Popen("echo 'yes' | ssh-keygen -t rsa -f {0}/{1} -C {1} -N '' ".format(savePath,username),
              stdout=PIPE,
              shell=True,
              stderr=PIPE
               )
        print(p.communicate())    

# Automated calibration for the GOTM/FABM model at Langtjern

## Pre-requisites

* gcloud command line utility
* google cloud account, you might need to be an admin for some of this to work, also this is not free
* ssh key setup in gitlab.au.dk
* fabric3 needs to be (pip) installed
* python modules listed in the first cell of this notebook

## Creating an instance on Google Cloud

In [2]:
instanceName = 'gotm-cal'
username = 'jose-luis'

createInstance = '''\
gcloud compute instances create {} \
--zone europe-west3-a \
--image-family debian-9 \
--image-project debian-cloud \
--machine-type n1-standard-2 \
'''
#--boot-disk-size 200GB \

deleteInstance = '''\
gcloud compute instances delete {} \
--zone europe-west-3a \
'''

listInstances = '''gcloud compute instances list'''

addSSHKeys = '''gcloud compute instances add-metadata {} --zone europe-west3-a --metadata-from-file ssh-keys={}'''

if (platform.system() == 'Linux'):
    keyDir = ('/home/jose-luis/.ssh/gotmKeys')

if (platform.system() == 'Windows'):
    keyDir = ('c:\\Users\\Guerrero\\Envs\\prognos_calibration\\prognos_calibration\\gotmKeys')
    

ip=''
instanceExists,ip = isInstance(instanceName)

if instanceExists:
    print('Instance {} is {}'.format(instanceName,ip) )

isStarted = False
if instanceExists and ip == 'TERMINATED' :
    callPopen('gcloud compute instances start {} --zone europe-west3-a'.format(instanceName))
    instanceExists,ip = isInstance(instanceName)
    isStarted = True
    print("Machine started and ip is {}".format(ip))

wasCreated=False
if not instanceExists and not isStarted:
    callPopen(createInstance.format(instanceName))
    wasCreated=True
    if os.path.exists(keyDir):
        shutil.rmtree(keyDir)
    os.mkdir(keyDir)
    generateSSHKey(username,keyDir)
    keyFile = os.path.join(keyDir,username + '.pub')
    text_prepender('{}/{}.pub'.format(keyDir,username), '{}:'.format(username) )
    callPopen(addSSHKeys.format(instanceName,keyDir + '/{}.pub'.format(username)))
    #callPopen('sed -i s/^{0}:// {1}/{0}.pub'.format(username,keyDir))
    replace(keyFile,r"^{}:".format(username),"")
    ip=isInstance(instanceName)[1]
        #callPopen('chmod 400 {}'.format(keyDir))
        
print("The ip of {} is {}".format(instanceName,ip))

gcloud compute instances create gotm-cal --zone europe-west3-a --image-family debian-9 --image-project debian-cloud --machine-type n1-standard-2 
NAME      ZONE            MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP     STATUS
gotm-cal  europe-west3-a  n1-standard-2               10.156.0.3   35.198.160.181  RUNNING
Created [https://www.googleapis.com/compute/v1/projects/nivacatchment/zones/europe-west3-a/instances/gotm-cal].
(b"Generating public/private rsa key pair.\nYour identification has been saved in /home/jose-luis/.ssh/gotmKeys/jose-luis.\nYour public key has been saved in /home/jose-luis/.ssh/gotmKeys/jose-luis.pub.\nThe key fingerprint is:\nSHA256:L+bjS4mdbZSkf+PYZwQl0LgY1eV6c989GY9JU2Oo1lE jose-luis\nThe key's randomart image is:\n+---[RSA 2048]----+\n|         .o= ..E |\n|        . . +.+  |\n|         o.. =.o.|\n|        .o..+.o..|\n|        S oo.o=..|\n|       o B.  o.*B|\n|      . B = o.oo=|\n|       +.o = .o .|\n|       .+o. oo   |\n+----[SHA256]-----+\n", b'')


## Updating fabfile.py with credentials and ip

In [12]:
if (platform.system == "Linux"):
    callPopen("sed -i s/^env\.hosts.*/env.hosts=\['{}']/ fabfile.py".format(ip))
    callPopen("sed -i s/^env\.user.*/env.user=\'{}\'/ fabfile.py".format(username))
    callPopen("sed -i s$^env\.key_filename.*$env\.key_filename='{}'$ fabfile.py".format(keyDir + '/' + username))
    callPopen("sed -i s/^env\.roledefs.*/env.roledefs={{\\'{}\\':[\\'{}\\'],/ fabfile.py".format('stage',ip))

if platform.syste    
replace("fabfile.py", r"^env\.hosts.*",         "env.hosts=['{}']".format(ip))
replace("fabfile.py", r"^env\.user.*",          "env.user='{}'".format(username))
replace("fabfile.py", r"^env\.key_filename.*",  "env.key_filename='{}'".format(os.path.join(keyDir,username)))
replace("fabfile.py", r"^env\.roledefs.*",      "env.roledefs={{'{}':['{}'],".format('stage',ip))

#Testing connection
#Adding key to remote machine
p = Popen("ssh -i {0}/{1} {1}@{2} -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no".format(keyDir,username,ip),shell=True,stdout=PIPE,stdin=PIPE)
print(p.communicate())

(b'Linux gotm-cal 4.9.0-7-amd64 #1 SMP Debian 4.9.110-3+deb9u1 (2018-08-03) x86_64\n\nThe programs included with the Debian GNU/Linux system are free software;\nthe exact distribution terms for each program are described in the\nindividual files in /usr/share/doc/*/copyright.\n\nDebian GNU/Linux comes with ABSOLUTELY NO WARRANTY, to the extent\npermitted by applicable law.\n', None)


## Setting up GOTM and FABM in the remote machine

In [ ]:
callPopen('fab testConnection')
callPopen('fab getUtilities')
callPopen('fab downloadModels') #note that the location of the key file is hardcoded in the getFABM function
callPopen('fab compileModels')
callPopen('fab getPROGNOS')
callPopen('fab testRun:/home/{}/PROGNOS/langtjern/langtjern.xml'.format(username))

fab testConnection
[35.198.160.181] Executing task 'testConnection'
[35.198.160.181] Executing task 'whoAmI'
[35.198.160.181] run: uname -a
[35.198.160.181] out: Linux gotm-cal 4.9.0-7-amd64 #1 SMP Debian 4.9.110-3+deb9u1 (2018-08-03) x86_64 GNU/Linux
[35.198.160.181] out: 

[35.198.160.181] run: whoami
[35.198.160.181] out: jose-luis
[35.198.160.181] out: 


Done.
Disconnecting from 35.198.160.181... done.
fab getUtilities
[35.198.160.181] Executing task 'getUtilities'
[35.198.160.181] Executing task 'update'
[35.198.160.181] Executing task 'updateMachine'
[35.198.160.181] run: sudo apt-get update
[35.198.160.181] out: 
[35.198.160.181] out: 0% [Working]
[35.198.160.181] out:             
[35.198.160.181] out: Ign:1 http://deb.debian.org/debian stretch InRelease
[35.198.160.181] out: 
[35.198.160.181] out: 0% [Waiting for headers] [Connecting to packages.cloud.google.com (172.217.22.7
[35.198.160.181] out:                                                                                

[35.198.160.181] out: 0% [6 InRelease gpgv 6,377 B]
[35.198.160.181] out:                              
[35.198.160.181] out: 0% [Working]
[35.198.160.181] out: 0% [7 InRelease gpgv 3,843 B] [Waiting for headers]
[35.198.160.181] out:                                                    
[35.198.160.181] out: Get:26 http://packages.cloud.google.com/apt cloud-sdk-stretch/main amd64 Packages [43.7 kB]
[35.198.160.181] out: 
[35.198.160.181] out: 0% [7 InRelease gpgv 3,843 B] [26 Packages 12.0 kB/43.7 kB 27%]
[35.198.160.181] out:                                                                
[35.198.160.181] out: 0% [7 InRelease gpgv 3,843 B]
[35.198.160.181] out: 0% [26 Packages store 0 B] [7 InRelease gpgv 3,843 B]
[35.198.160.181] out:                                                      
[35.198.160.181] out: 0% [7 InRelease gpgv 3,843 B]
[35.198.160.181] out:                              
[35.198.160.181] out: 0% [Working]
[35.198.160.181] out: 0% [10 InRelease gpgv 3,876 B]
[35.198.

[35.198.160.181] out: 29% [Waiting for headers]
[35.198.160.181] out:                          
[35.198.160.181] out: Get:17 http://deb.debian.org/debian stretch/main amd64 sgml-base all 1.29 [14.8 kB]
[35.198.160.181] out: 
[35.198.160.181] out:                          
[35.198.160.181] out: Get:18 http://deb.debian.org/debian stretch/main amd64 bzip2 amd64 1.0.6-8.1 [47.5 kB]
[35.198.160.181] out: 
[35.198.160.181] out: 30% [18 bzip2 47.5 kB/47.5 kB 100%]
[35.198.160.181] out:                                    
[35.198.160.181] out: 30% [Working]
[35.198.160.181] out:              
[35.198.160.181] out: Get:19 http://deb.debian.org/debian stretch/main amd64 less amd64 481-2.1 [126 kB]
[35.198.160.181] out: 
[35.198.160.181] out: 30% [19 less 52.4 kB/126 kB 41%]
[35.198.160.181] out:                                 
[35.198.160.181] out: 30% [Waiting for headers]
[35.198.160.181] out:                          
[35.198.160.181] out: Get:20 http://deb.debian.org/debian stretch/main am

[35.198.160.181] out: 70% [48 gfortran-6 65.5 kB/6,916 kB 1%]
[35.198.160.181] out:                                        
[35.198.160.181] out: 76% [Waiting for headers]
[35.198.160.181] out:                          
[35.198.160.181] out: Get:49 http://deb.debian.org/debian stretch/main amd64 gfortran amd64 4:6.3.0-4 [1,356 B]
[35.198.160.181] out: 
[35.198.160.181] out:                          
[35.198.160.181] out: Get:50 http://deb.debian.org/debian stretch/main amd64 hdf5-helpers amd64 1.10.0-patch1+docs-3+deb9u1 [36.0 kB]
[35.198.160.181] out: 
[35.198.160.181] out: 76% [50 hdf5-helpers 4,096 B/36.0 kB 11%]
[35.198.160.181] out:                                          
[35.198.160.181] out: 77% [Working]
[35.198.160.181] out:              
[35.198.160.181] out: Get:51 http://deb.debian.org/debian stretch/main amd64 libaec0 amd64 0.3.2-1 [18.5 kB]
[35.198.160.181] out: 
[35.198.160.181] out: 77% [51 libaec0 18.5 kB/18.5 kB 100%]
[35.198.160.181] out:                           

[35.198.160.181] out: 
[35.198.160.181] out: 97% [76 rename 12.5 kB/12.5 kB 100%]
[35.198.160.181] out:                                     
[35.198.160.181] out: 97% [Waiting for headers]
[35.198.160.181] out:                          
[35.198.160.181] out: Get:77 http://deb.debian.org/debian stretch/main amd64 rsync amd64 3.1.2-1+deb9u1 [393 kB]
[35.198.160.181] out: 
[35.198.160.181] out: 97% [77 rsync 24.2 kB/393 kB 6%]
[35.198.160.181] out:                                 
[35.198.160.181] out: 98% [Waiting for headers]
[35.198.160.181] out:                          
[35.198.160.181] out: Get:78 http://deb.debian.org/debian stretch/main amd64 shared-mime-info amd64 1.8-1+deb9u1 [731 kB]
[35.198.160.181] out: 
[35.198.160.181] out: 98% [78 shared-mime-info 24.0 kB/731 kB 3%]
[35.198.160.181] out:                                            
[35.198.160.181] out: 99% [Waiting for headers]
[35.198.160.181] out:                          
[35.198.160.181] out: Get:79 http://deb.debian.o

...

[35.198.160.181] out:     87% |████████████████████████████    | 12.1MB 56.5MB/s eta 0:00:01
[35.198.160.181] out:     87% |████████████████████████████    | 12.1MB 57.2MB/s eta 0:00:01
[35.198.160.181] out:     87% |████████████████████████████    | 12.1MB 56.7MB/s eta 0:00:01
[35.198.160.181] out:     87% |████████████████████████████▏   | 12.2MB 56.9MB/s eta 0:00:01
[35.198.160.181] out:     87% |████████████████████████████▏   | 12.2MB 56.9MB/s eta 0:00:01
[35.198.160.181] out:     88% |████████████████████████████▏   | 12.2MB 57.0MB/s eta 0:00:01
[35.198.160.181] out:     88% |████████████████████████████▏   | 12.2MB 55.2MB/s eta 0:00:01
[35.198.160.181] out:     88% |████████████████████████████▏   | 12.2MB 57.9MB/s eta 0:00:01
[35.198.160.181] out:     88% |████████████████████████████▎   | 12.2MB 56.4MB/s eta 0:00:01
[35.198.160.181] out:     88% |████████████████████████████▎   | 12.2MB 55.7MB/s eta 0:00:01
[35.198.160.181] out:     88% |████████████████████████████▎   | 12.2M

[35.198.160.181] out:     95% |██████████████████████████████▌ | 13.2MB 59.0MB/s eta 0:00:01
[35.198.160.181] out:     95% |██████████████████████████████▌ | 13.2MB 57.8MB/s eta 0:00:01
[35.198.160.181] out:     95% |██████████████████████████████▌ | 13.2MB 56.7MB/s eta 0:00:01
[35.198.160.181] out:     95% |██████████████████████████████▋ | 13.2MB 57.3MB/s eta 0:00:01
[35.198.160.181] out:     95% |██████████████████████████████▋ | 13.2MB 57.5MB/s eta 0:00:01
[35.198.160.181] out:     95% |██████████████████████████████▋ | 13.2MB 58.7MB/s eta 0:00:01
[35.198.160.181] out:     95% |██████████████████████████████▋ | 13.2MB 59.3MB/s eta 0:00:01
[35.198.160.181] out:     95% |██████████████████████████████▋ | 13.3MB 56.6MB/s eta 0:00:01
[35.198.160.181] out:     95% |██████████████████████████████▊ | 13.3MB 56.9MB/s eta 0:00:01
[35.198.160.181] out:     95% |██████████████████████████████▊ | 13.3MB 57.0MB/s eta 0:00:01
[35.198.160.181] out:     96% |██████████████████████████████▊ | 13.3M

[35.198.160.181] out:     31% |██████████                      | 296kB 50.4MB/s eta 0:00:01
[35.198.160.181] out:     32% |██████████▎                     | 307kB 54.1MB/s eta 0:00:01
[35.198.160.181] out:     33% |██████████▊                     | 317kB 53.0MB/s eta 0:00:01
[35.198.160.181] out:     34% |███████████                     | 327kB 52.4MB/s eta 0:00:01
[35.198.160.181] out:     35% |███████████▍                    | 337kB 52.7MB/s eta 0:00:01
[35.198.160.181] out:     36% |███████████▊                    | 348kB 52.9MB/s eta 0:00:01
[35.198.160.181] out:     37% |████████████                    | 358kB 53.0MB/s eta 0:00:01
[35.198.160.181] out:     38% |████████████▍                   | 368kB 55.9MB/s eta 0:00:01
[35.198.160.181] out:     39% |████████████▊                   | 378kB 55.6MB/s eta 0:00:01
[35.198.160.181] out:     40% |█████████████                   | 389kB 56.1MB/s eta 0:00:01
[35.198.160.181] out:     41% |█████████████▍                  | 399kB 57.8MB/s 

[35.198.160.181] out: 
[35.198.160.181] out:     2% |▋                               | 10kB 37.8MB/s eta 0:00:01
[35.198.160.181] out:     4% |█▎                              | 20kB 40.0MB/s eta 0:00:01
[35.198.160.181] out:     6% |██                              | 30kB 44.2MB/s eta 0:00:01
[35.198.160.181] out:     8% |██▋                             | 40kB 46.6MB/s eta 0:00:01
[35.198.160.181] out:     10% |███▏                            | 51kB 45.3MB/s eta 0:00:01
[35.198.160.181] out:     12% |███▉                            | 61kB 46.5MB/s eta 0:00:01
[35.198.160.181] out:     14% |████▌                           | 71kB 47.4MB/s eta 0:00:01
[35.198.160.181] out:     16% |█████▏                          | 81kB 48.8MB/s eta 0:00:01
[35.198.160.181] out:     18% |█████▊                          | 92kB 49.4MB/s eta 0:00:01
[35.198.160.181] out:     20% |██████▍                         | 102kB 49.2MB/s eta 0:00:01
[35.198.160.181] out:     22% |███████                         | 112kB

[35.198.160.181] out: Warning: Permanently added 'gitlab.au.dk,185.45.23.235' (ECDSA) to the list of known hosts.
[35.198.160.181] out: 
